In [22]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv

load_dotenv()

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [23]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("./us_census")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'source': 'us_census/acsbr-015.pdf', 'page': 0}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3')

In [24]:
len(final_documents)

316

In [25]:
## Embedding Using Huggingface
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

In [26]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-8.46568570e-02 -1.19099133e-02 -3.37892696e-02  2.94559319e-02
  5.19159958e-02  5.73839359e-02 -4.10017595e-02  2.74267979e-02
 -1.05128214e-01 -1.58056244e-02  7.94858858e-02  5.64318411e-02
 -1.31765660e-02 -3.41543816e-02  5.81603171e-03  4.72547710e-02
 -1.30747100e-02  3.12988390e-03 -3.44225727e-02  3.08406446e-02
 -4.09086309e-02  3.52738015e-02 -2.43761651e-02 -4.35831398e-02
  2.41503455e-02  1.31986784e-02 -4.84452490e-03  1.92347504e-02
 -5.43912686e-02 -1.42735019e-01  5.15527138e-03  2.93115824e-02
 -5.60810789e-02 -8.53536837e-03  3.14141475e-02  2.76736654e-02
 -2.06188485e-02  8.24231207e-02  4.15425599e-02  5.79655021e-02
 -3.71587090e-02  6.26161881e-03 -2.41390113e-02 -5.61793661e-03
 -2.51715276e-02  5.04971622e-03 -2.52801217e-02 -2.91943806e-03
 -8.24045856e-03 -5.69604337e-02  2.30822787e-02 -5.54220518e-03
  5.11555672e-02  6.09937720e-02  6.49766549e-02 -5.38514033e-02
  2.19109617e-02 -2.54194122e-02 -4.49223146e-02  4.22459133e-02
  4.75252233e-02  7.23247

In [27]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [28]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [29]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x110f809a0> search_kwargs={'k': 3}


In [38]:
from langchain_community.llms import HuggingFaceHub
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=os.getenv("HUGGINGFACEHUB_API_TOKEN")
hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance rating?"
hf.invoke(query)

'What is the health insurance rating?\n\nThe health insurance rating is a measure of the quality of health insurance plans. It is based on a number of factors, including the number of complaints filed against the plan, the number of claims paid, and the number of claims denied.\n\nWhat are the benefits of a health insurance rating?\n\nThe health insurance rating can help consumers make informed decisions about their health insurance plans. It can also help insurers identify areas where they need to improve their services.\n\nHow is the'

In [39]:
query="What is the health insurance coverage?"
hf.invoke(query)

'What is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer undertakes to pay the insured for the costs incurred in the event of illness or accident. The insured undertakes to pay the premiums.\n\nWhat is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer undertakes to pay the insured for the costs incurred in the event of'